# Taller 1 - Red de Monitoreo de Calidad del Aire de Bogota.
http://rmcab.ambientebogota.gov.co/Report/stationreport

## Decripción del conjunto de datos

# 1. Instalar Librerias

In [39]:
#!pip install --upgrade pip # se actualiza pip, en caso de que no lo tenga actualizado
#!pip install psycopg2-binary # Psycopg es un adaptador de base de datos PostgreSQL
#!pip install pandas 
#!pip install openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 965.1 kB/s eta 0:00:00:0100:010m


## 1.1 Importar librerias

In [2]:
import pandas as pd
from os import walk
import os, re   #expresiones regulares

# 2. Procesar conjunto de datos (Preprocesssing-dataset)

## 2.1 Unir todos los conjuntos de datos de extensión xlsx en un arreglo 


In [3]:
# unir todos los conjuntos de datos de extensión xlsx en un arreglo 
arregloDeDataSets = []
for (dirpath, dirnames, filenames) in walk('data/raw/'):
    arregloDeDataSets.extend(filenames)
    break
arregloDeDataSets

['ca_7MA_StationsReport_20223129920.xlsx',
 'ca_BOL_StationsReport_20223129119.xlsx',
 'ca_CBV_StationsReport_20223129333.xlsx',
 'ca_CDAR_StationsReport_20223129247.xlsx',
 'ca_COL_StationsReport_20223129424.xlsx',
 'ca_CSE_StationsReport_20223129152.xlsx',
 'ca_FTB_StationsReport_20223129456.xlsx',
 'ca_GYR_StationsReport_20223129531.xlsx',
 'ca_JAZ_StationsReport_20223129614.xlsx',
 'ca_KEN_StationsReport_2022312979.xlsx',
 'ca_LFR_StationsReport_20223129750.xlsx',
 'ca_MAM_StationsReport_20223129831.xlsx',
 'ca_MOV2_StationsReport_202231291026.xlsx',
 'ca_PTE_StationsReport_20223129110.xlsx',
 'ca_SCR_StationsReport_202231291142.xlsx',
 'ca_SUB_StationsReport_202231291211.xlsx',
 'ca_TUN_StationsReport_202231291241.xlsx',
 'ca_USM_StationsReport_202231291350.xlsx',
 'ca_USQ_StationsReport_202231291319.xlsx']

## 2.2  Concatenar los datsets en uno solo 


In [4]:
path = os.path.abspath('data/raw/') 
files = os.listdir(path)  

df = pd.DataFrame()
for file in files:
    if file.endswith('.xlsx'):
        dft = pd.read_excel('data/raw/'+file)
        dft['Station'] = re.search(r'_(.*?)_', str(file)).group(1)
        df = df.append(dft[1:], ignore_index=True) 
df.head() 

/tmp/ipykernel_182/3607398716.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dft[1:], ignore_index=True)
/tmp/ipykernel_182/3607398716.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dft[1:], ignore_index=True)
/tmp/ipykernel_182/3607398716.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dft[1:], ignore_index=True)
/tmp/ipykernel_182/3607398716.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dft[1:], ignore_index=True)
/tmp/ipykernel_182/3607398716.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Us

,DateTime,PM10,CO,NO,NO2,NOX,Vel Viento,Dir Viento,Temperatura,Presion Baro,...,CO2,SO2 Envea,Vel Viento 10M,Dir Viento 10M,Temperatura 8M,Temperatura 20M,PM2.5 Flow,PM10 Flow,HR.1,Canal no activo
0,01-01-2021 01:00,52.6,----,53.384,3.428,56.817,0.4,360,14,562,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01-01-2021 02:00,77.9,----,49.105,3.42,52.525,0,171,13.9,562,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01-01-2021 03:00,58.5,----,47.284,4.062,51.346,0.1,170,13.8,561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01-01-2021 04:00,57.2,----,46.059,3.606,49.664,0.1,235,13.3,561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01-01-2021 05:00,53.5,----,39.851,1.625,41.475,0.1,312,13.3,562,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.3 Comprobar los valores nulos por columnas

In [5]:
#Contar cuantos valores nulos hay por cada columna
df.isnull().sum()

DateTime                0
PM10                    0
CO                      0
NO                   8760
NO2                  8760
NOX                  8760
Vel Viento          26280
Dir Viento          26280
Temperatura         17520
Presion Baro        70080
Rad Solar           61320
Precipitacion       61320
HR                  52560
PM2.5                   0
Station                 0
SO2                 43800
OZONO                8760
CO2                148920
SO2 Envea          157680
Vel Viento 10M     157680
Dir Viento 10M     157680
Temperatura 8M     157680
Temperatura 20M    157680
PM2.5 Flow         157680
PM10 Flow          157680
HR.1               157680
Canal no activo    157680
dtype: int64

## 2.4 Organizar y guardar en un nuevo df las columnas significativas

In [6]:
#Organizar y guardar en un nuevo df las columnas significativas
df = df[['PM10','PM2.5','NO','NO2','NOX','CO','OZONO','Station', 'DateTime']]
df.head()

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime
0,52.6,37,53.384,3.428,56.817,----,NaN,7MA,01-01-2021 01:00
1,77.9,61,49.105,3.42,52.525,----,NaN,7MA,01-01-2021 02:00
2,58.5,48,47.284,4.062,51.346,----,NaN,7MA,01-01-2021 03:00
3,57.2,44,46.059,3.606,49.664,----,NaN,7MA,01-01-2021 04:00
4,53.5,39,39.851,1.625,41.475,----,NaN,7MA,01-01-2021 05:00


## 2.5 Convertir valores a numericos y sino poner NaN

In [7]:
#Aquí su código 

In [8]:
#df.isnull().sum()

## 2.6 Ver el tamaño del dataset 

In [9]:
#ver el tamaño del conjunto de datos 
#Aquí su código 

(166440, 9)

## 2.7 Crear una nueva columna con filtro (False o True)

In [10]:
# True is greater than 12, bad (Pure, Not Pure)
df['Status'] = df['PM2.5']>12
df.Status.value_counts()
df.head()

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime,Status
0,52.6,37.0,53.384,3.428,56.817,NaN,NaN,7MA,01-01-2021 01:00,True
1,77.9,61.0,49.105,3.420,52.525,NaN,NaN,7MA,01-01-2021 02:00,True
2,58.5,48.0,47.284,4.062,51.346,NaN,NaN,7MA,01-01-2021 03:00,True
3,57.2,44.0,46.059,3.606,49.664,NaN,NaN,7MA,01-01-2021 04:00,True
4,53.5,39.0,39.851,1.625,41.475,NaN,NaN,7MA,01-01-2021 05:00,True


## 2.8 Guardar todo el proceso en un nuevo dataset

In [11]:
# we need another column to specify stations
#df.to_csv("data/clean_data_final.csv", index = False)
#Aquí su código 

# 3. Integrar nuevo dataset (Stations)

## 3.1 Cargar dataset de stations

In [12]:
#cargar el df dataset_with_missing.csv   data/dataset_with_missing.csv'
#cargar el df stations_loc.csv data/stations_loc.csv')

#Aquí su código 

,estacion,Sigla,Latitud,Longitud,Altitud (m),Altura (m),Localidad,Tipo de zona,Tipo de estación,Dirección
0,guaymaral,GYR,"4°47'01.5""N","74°02'38.9""W",2580,0,Suba,Sub urbana,De fondo,Autopista Norte # 205-59
1,usaquen,USQ,"4°42'37.26""N","74°1'49.50""W",2570,10,Usaquén,Urbana,De fondo,Carrera 7B Bis # 132-11
2,suba,SUB,"4°45'40.49""N","74° 5'36.46""W",2571,6,Suba,Sub urbana,De fondo,Carrera 111 # 159A-61
3,bolivia,BOL,"4°44'08.9""N","74°07'33.2""W",2574,0,Engativá,Sub urbana,De fondo,Avenida Calle 80 # 121-98
4,las_ferias,LFR,"4°41'26.52""N","74°4'56.94""W",2552,0,Engativá,Urbana,De tráfico,Avenida Calle 80 # 69Q-50
5,centro_alto_rendimiento,CDAR,"4°39'30.48""N","74°5'2.28""W",2552,0,Engativá,Urbana,De tráfico,Calle 63 # 59A-06
6,movil_7ma,7MA,"4°38'42.7""N","74°03'41.6""W",2583,0,Chapinero,Urbana,De tráfico,Carrera 7 con calle 60
7,minambiente,MAM,"4°37'31.75""N","74°4'1.13""W",2621,15,Santa Fe,Urbana,De tráfico,Calle 37 # 8-40
8,fontibon,FTB,"4°40'41.67""N","74°8'37.75""W",2551,11,Fontibón,Urbana,De tráfico,Carrera 104 # 20 C - 31
9,pueste_aranda,PTE,"4°37'54.36""N","74°7'2.94""W",2590,10,Puente Aranda,Urbana,Industrial,Calle 10 # 65-28


### 3.1.1 Seleccionar algunas columnas del dataset stations

In [14]:
stations = stations[['Sigla', 'Latitud', 'Longitud']] 


### 3.1.2 Cambiar el nombre de una columna 

In [15]:
stations = stations.rename(columns={'Sigla': 'Station'}) #Columna Sigla por Station
stations.head()

,Station,Latitud,Longitud
0,GYR,"4°47'01.5""N","74°02'38.9""W"
1,USQ,"4°42'37.26""N","74°1'49.50""W"
2,SUB,"4°45'40.49""N","74° 5'36.46""W"
3,BOL,"4°44'08.9""N","74°07'33.2""W"
4,LFR,"4°41'26.52""N","74°4'56.94""W"


## 3.2 Convertir las coordenadas de texto a decimales

In [16]:
import re

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'S' or direction == 'W':
        dd *= -1
    return dd;

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

def parse_dms(coor):
    parts = re.split('[^\d\w]+', coor)
    dec_coor = dms2dd(parts[0], parts[1], float(parts[2]+'.'+parts[2]), parts[4])
    return dec_coor

'''
def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'E' or direction == 'S':
        dd *= -1
    return dd;

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

def parse_dms(dms):
    parts = re.split('[^\d\w]+', dms)
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])
 
    return (lat)
'''

"\ndef dms2dd(degrees, minutes, seconds, direction):\n    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);\n    if direction == 'E' or direction == 'S':\n        dd *= -1\n    return dd;\n\ndef dd2dms(deg):\n    d = int(deg)\n    md = abs(deg - d) * 60\n    m = int(md)\n    sd = (md - m) * 60\n    return [d, m, sd]\n\ndef parse_dms(dms):\n    parts = re.split('[^\\d\\w]+', dms)\n    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])\n \n    return (lat)\n"

In [17]:
#Aplicar la funcion parse_dms a la columna Latitud y Longitud
stations['Latitud'] = stations['Latitud'].apply(parse_dms)
stations['Longitud'] = stations['Longitud'].apply(parse_dms)
stations.head()

,Station,Latitud,Longitud
0,GYR,4.783614,-74.043994
1,USQ,4.710381,-74.030414
2,SUB,4.761222,-74.093433
3,BOL,4.735578,-74.125925
4,LFR,4.690628,-74.082378


## 3.3 Integrar el dataset df con el dataset stations

In [18]:
df = pd.merge(df, stations, on='Station', how='inner')
df.head(3)

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime,Status,Latitud,Longitud
0,52.6,37.0,53.384,3.428,56.817,NaN,NaN,7MA,01-01-2021 01:00,True,4.645117,-74.061503
1,77.9,61.0,49.105,3.420,52.525,NaN,NaN,7MA,01-01-2021 02:00,True,4.645117,-74.061503
2,58.5,48.0,47.284,4.062,51.346,NaN,NaN,7MA,01-01-2021 03:00,True,4.645117,-74.061503


## 3.4 Convertir hora (DateTime)

In [19]:
#Puede ver que en la columna 'DateTime', 
#la información sobre la fecha y la hora se dan juntas. 
#Por lo tanto, extraerá la información de tiempo.

def replace24(datetimex):
    return datetimex.replace('24:00', '00:00') #cambia de horarios

In [20]:
# Esta celda extraerá información de la columna 'datetime' y 
#generará columnas de meses, días o semanas y horas
df['DateTime'] = df['DateTime'].apply(replace24)
df['DateTime'] = pd.to_datetime(df['DateTime'], dayfirst=True)
df['month'] = pd.DatetimeIndex(df['DateTime']).month
df['day_week'] = pd.DatetimeIndex(df['DateTime']).weekday
df['day_month'] = pd.DatetimeIndex(df['DateTime']).day
df['hour'] = pd.DatetimeIndex(df['DateTime']).hour
df.loc[df['hour']==0,'hour'] = 24

In [21]:
df

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime,Status,Latitud,Longitud,month,day_week,day_month,hour
0,52.6,37.0,53.384,3.428,56.817,NaN,NaN,7MA,2021-01-01 01:00:00,True,4.645117,-74.061503,1,4,1,1
1,77.9,61.0,49.105,3.420,52.525,NaN,NaN,7MA,2021-01-01 02:00:00,True,4.645117,-74.061503,1,4,1,2
2,58.5,48.0,47.284,4.062,51.346,NaN,NaN,7MA,2021-01-01 03:00:00,True,4.645117,-74.061503,1,4,1,3
3,57.2,44.0,46.059,3.606,49.664,NaN,NaN,7MA,2021-01-01 04:00:00,True,4.645117,-74.061503,1,4,1,4
4,53.5,39.0,39.851,1.625,41.475,NaN,NaN,7MA,2021-01-01 05:00:00,True,4.645117,-74.061503,1,4,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166435,25.3,18.0,1.404,3.688,5.091,0.26467,29.773,USQ,2021-12-31 20:00:00,True,4.710381,-74.030414,12,4,31,20
166436,27.7,15.0,0.244,3.205,3.449,0.27138,28.059,USQ,2021-12-31 21:00:00,True,4.710381,-74.030414,12,4,31,21
166437,25.3,20.0,0.880,4.648,5.529,0.30159,25.561,USQ,2021-12-31 22:00:00,True,4.710381,-74.030414,12,4,31,22
166438,24.0,17.0,1.697,8.596,10.293,0.34977,20.539,USQ,2021-12-31 23:00:00,True,4.710381,-74.030414,12,4,31,23


## 3.5 Guardar todo el proceso en un nuevo dataset

In [22]:
df.to_csv("data/dataset_with_geo_missing.csv", index = False)

# 4. Limpieza de datos

In [23]:
# Varias columnas tienen valores perdidos en el dataset

#Aquí su código 

PM10         20014
PM2.5        15312
NO           27664
NO2          27662
NOX          27668
CO           31238
OZONO        32132
Station          0
DateTime         0
Status           0
Latitud          0
Longitud         0
month            0
day_week         0
day_month        0
hour             0
dtype: int64

## 4.1 Remplazar los valores NaN en cada columna con su media

In [24]:
df['PM10'].fillna((df['PM10'].mean()), inplace=True)

#hagalo para PM2.5, CO, NO, NO2, NOx, OZONO
#Aquí su código 

In [25]:
# compruebe --- ya no hay valores perdidos en el dataset

#df.isnull().sum()

PM10         0
PM2.5        0
NO           0
NO2          0
NOX          0
CO           0
OZONO        0
Station      0
DateTime     0
Status       0
Latitud      0
Longitud     0
month        0
day_week     0
day_month    0
hour         0
dtype: int64

## 4.2 Guardar todo el proceso en un nuevo dataset

In [26]:
df.to_csv("data/dataset_final_clean_mean.csv", index = False)

# 5. Creando el esquema de la bodega de datos 

In [27]:
df = pd.read_csv('data/dataset_final_clean_mean.csv')
df.head(10)

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime,Status,Latitud,Longitud,month,day_week,day_month,hour
0,52.6,37.0,53.384,3.428,56.817,0.691233,11.782188,7MA,2021-01-01 01:00:00,True,4.645117,-74.061503,1,4,1,1
1,77.9,61.0,49.105,3.420,52.525,0.691233,11.782188,7MA,2021-01-01 02:00:00,True,4.645117,-74.061503,1,4,1,2
2,58.5,48.0,47.284,4.062,51.346,0.691233,11.782188,7MA,2021-01-01 03:00:00,True,4.645117,-74.061503,1,4,1,3
3,57.2,44.0,46.059,3.606,49.664,0.691233,11.782188,7MA,2021-01-01 04:00:00,True,4.645117,-74.061503,1,4,1,4
4,53.5,39.0,39.851,1.625,41.475,0.691233,11.782188,7MA,2021-01-01 05:00:00,True,4.645117,-74.061503,1,4,1,5
5,22.3,15.0,39.956,0.677,40.633,0.691233,11.782188,7MA,2021-01-01 06:00:00,True,4.645117,-74.061503,1,4,1,6
6,18.3,8.0,40.870,1.069,41.940,0.691233,11.782188,7MA,2021-01-01 07:00:00,False,4.645117,-74.061503,1,4,1,7
7,23.2,10.0,42.630,1.615,44.245,0.691233,11.782188,7MA,2021-01-01 08:00:00,False,4.645117,-74.061503,1,4,1,8
8,24.4,12.0,38.317,12.145,50.463,0.691233,11.782188,7MA,2021-01-01 09:00:00,False,4.645117,-74.061503,1,4,1,9
9,15.2,9.0,37.499,10.979,48.478,0.691233,11.782188,7MA,2021-01-01 10:00:00,False,4.645117,-74.061503,1,4,1,10


## 5.1 Se crea un nuevo DataFrame con los datos de los polutantes

In [28]:
df_polutante=pd.DataFrame(df, columns=["PM10", "PM2.5", "NO", "NO2", "NOX", "CO", "OZONO"]) # se crea un nuevo DataFrame unicamente con los polutantes
df['id_polutante'] = df.index+1 # se crea una nueva columna que identificará a cada polutante
df_polutante

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO
0,52.6,37.0,53.384,3.428,56.817,0.691233,11.782188
1,77.9,61.0,49.105,3.420,52.525,0.691233,11.782188
2,58.5,48.0,47.284,4.062,51.346,0.691233,11.782188
3,57.2,44.0,46.059,3.606,49.664,0.691233,11.782188
4,53.5,39.0,39.851,1.625,41.475,0.691233,11.782188
...,...,...,...,...,...,...,...
166435,25.3,18.0,1.404,3.688,5.091,0.264670,29.773000
166436,27.7,15.0,0.244,3.205,3.449,0.271380,28.059000
166437,25.3,20.0,0.880,4.648,5.529,0.301590,25.561000
166438,24.0,17.0,1.697,8.596,10.293,0.349770,20.539000


## 5.2 Se crea un nuevo dataset con los datos de las estaciones y se eliminan las filas repetidas

In [29]:
df_estacion=pd.DataFrame(df, columns=["Name", "Station", "Localidad", "Latitud", "Longitud"]) # se crea un nuevo DataFrame unicamente con los datos de las estaciones
df_estacion = df_estacion.drop_duplicates() # se eliminan las filas repetidas
df_estacion = df_estacion.rename(columns={'Station': 'Sigla'}) # se cambia el nombre a la columna Station por Sigla
df_estacion

,Name,Sigla,Localidad,Latitud,Longitud
0,NaN,7MA,NaN,4.645117,-74.061503
8760,NaN,BOL,NaN,4.735578,-74.125925
17520,NaN,CBV,NaN,4.577889,-74.166272
26280,NaN,CDAR,NaN,4.658417,-74.083944
35040,NaN,COL,NaN,4.736981,-74.069472
43800,NaN,CSE,NaN,4.595958,-74.148483
52560,NaN,FTB,NaN,4.678169,-74.143714
61320,NaN,GYR,NaN,4.783614,-74.043994
70080,NaN,JAZ,NaN,4.608417,-74.114869
78840,NaN,KEN,NaN,4.625083,-74.161222


In [30]:
# se remplaza la columna sigla, dejando unicamente los datos unicos y luego se le cambia el nombre
# a la misma por id_estacion
count = 1
for index, row in df_estacion.iterrows():
    df = df.replace({row["Sigla"]: count})
    count += 1
df = df.rename(columns={'Station': 'id_estacion'})
df

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,id_estacion,DateTime,Status,Latitud,Longitud,month,day_week,day_month,hour,id_polutante
0,52.6,37.0,53.384,3.428,56.817,0.691233,11.782188,1,2021-01-01 01:00:00,True,4.645117,-74.061503,1,4,1,1,1
1,77.9,61.0,49.105,3.420,52.525,0.691233,11.782188,1,2021-01-01 02:00:00,True,4.645117,-74.061503,1,4,1,2,2
2,58.5,48.0,47.284,4.062,51.346,0.691233,11.782188,1,2021-01-01 03:00:00,True,4.645117,-74.061503,1,4,1,3,3
3,57.2,44.0,46.059,3.606,49.664,0.691233,11.782188,1,2021-01-01 04:00:00,True,4.645117,-74.061503,1,4,1,4,4
4,53.5,39.0,39.851,1.625,41.475,0.691233,11.782188,1,2021-01-01 05:00:00,True,4.645117,-74.061503,1,4,1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166435,25.3,18.0,1.404,3.688,5.091,0.264670,29.773000,19,2021-12-31 20:00:00,True,4.710381,-74.030414,12,4,31,20,166436
166436,27.7,15.0,0.244,3.205,3.449,0.271380,28.059000,19,2021-12-31 21:00:00,True,4.710381,-74.030414,12,4,31,21,166437
166437,25.3,20.0,0.880,4.648,5.529,0.301590,25.561000,19,2021-12-31 22:00:00,True,4.710381,-74.030414,12,4,31,22,166438
166438,24.0,17.0,1.697,8.596,10.293,0.349770,20.539000,19,2021-12-31 23:00:00,True,4.710381,-74.030414,12,4,31,23,166439


## 5.3 Se crea un nuevo DataFrame con los datos de las fechas.

In [31]:
# se crea un nuevo DataFrame y se eliminan los valores repetidos
df_fecha=pd.DataFrame(df, columns=["DateTime"]) # se crea un nuevo DataFrame con las fechas y horas 
#df_fecha["DateTime"].value_counts() # !TODO Borrar?
df_fecha = df_fecha.drop_duplicates() # se eliminan las filas repetidas
df_fecha

,DateTime
0,2021-01-01 01:00:00
1,2021-01-01 02:00:00
2,2021-01-01 03:00:00
3,2021-01-01 04:00:00
4,2021-01-01 05:00:00
...,...
8755,2021-12-31 20:00:00
8756,2021-12-31 21:00:00
8757,2021-12-31 22:00:00
8758,2021-12-31 23:00:00


In [32]:
import re # libreria para el uso de expresiones regulares
from pandas.tseries.holiday import Holiday, AbstractHolidayCalendar
from datetime import datetime

# definimos los días festivo para el año en que se tomaron los datos
class EsBusinessCalendar(AbstractHolidayCalendar):
   rules = [ # se definen los festivos, teniendo en cuenta los horarios festivos para el año de 2021 en Colombia
     Holiday('Año Nuevo', month=1, day=1),
     Holiday('Día de los Reyes Magos', month=1, day=11),
     Holiday('Día de San José', month=3, day=22),
     Holiday('Jueves Santo', month=4, day=1),
     Holiday('Viernes Santo', month=4, day=2),
     Holiday('Día del Trabajador', month=5, day=1),
     Holiday('Día de la Ascensión', month=5, day=17),
     Holiday('Corpus Christi', month=6, day=7),
     Holiday('Sagrado Corazón', month=6, day=14),
     Holiday('San Pedro y San Pablo ', month=7, day=5),
     Holiday('Día de la Independencia', month=7, day=20),
     Holiday('Batalla de Boyacá', month=8, day=7),
     Holiday('Asunción de la Virgen', month=8, day=16),
     Holiday('Celebración del Día de la Raza', month=10, day=18),
     Holiday('Día de todos los Santos', month=11, day=1),
     Holiday('Independencia de Cartagena', month=11, day=15),
     Holiday('Inmaculada Concepción', month=12, day=8),    
     Holiday('Navidad', month=12, day=25)
   ]

calendar_festivos = EsBusinessCalendar() # se instancia el objeto
calendar_festivos = calendar_festivos.holidays(start='2021-01-01', end='2021-12-31') # se define el rango de tiempo en que se tendran en cuenta los festivos (anho 2021)

dias = []
meses = []
anhos = []
horas = []
fin_semana = []
festivo = []


for index, row in df_fecha.iterrows(): # se recorre cada fila del DataFrame
    list_fecha = re.split("[\-\s]", row['DateTime']) # 
    dias.append(list_fecha[2])
    meses.append(list_fecha[1])
    anhos.append(list_fecha[0])
    horas.append(list_fecha[3])

    hora = row['DateTime'].split(" ")
    hora[0] = datetime.strptime(hora[0], '%Y-%m-%d')
    if hora[0].weekday() < 5 : # condicción para determinar si el día está comprendido entre lues-viernes
        fin_semana.append(False)
    else:
        fin_semana.append(True)
    if hora[0] in calendar_festivos: # condicción para determinar si el día hace parte de los días festivos del año
        festivo.append(True)
    else: 
        festivo.append(False)

dict_fechas = { 'dia': dias, 'mes': meses, 'anho': anhos, 'hora': horas, 'fin_semana': fin_semana, 'festivo': festivo}
df_fechas = pd.DataFrame(data=dict_fechas)
df_fechas


,dia,mes,anho,hora,fin_semana,festivo
0,01,01,2021,01:00:00,False,True
1,01,01,2021,02:00:00,False,True
2,01,01,2021,03:00:00,False,True
3,01,01,2021,04:00:00,False,True
4,01,01,2021,05:00:00,False,True
...,...,...,...,...,...,...
8755,31,12,2021,20:00:00,False,False
8756,31,12,2021,21:00:00,False,False
8757,31,12,2021,22:00:00,False,False
8758,31,12,2021,23:00:00,False,False


## 5.4 Creación de la columna id_tiempo para identificar cada una de las fechas en el DF principal

In [35]:
count = 1
# se recorre cada fecha de df_fecha
for index, row in df_fecha.iterrows(): 
# por cada fila de df_fecha se crea un nuevo identificador en el DataFrame principal
    df.loc[df['DateTime'] == row["DateTime"], 'DateTime'] = count     
    count += 1
df = df.rename(columns={'DateTime': 'id_tiempo'})
df

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,id_estacion,id_tiempo,Status,Latitud,Longitud,month,day_week,day_month,hour,id_polutante
0,52.6,37.0,53.384,3.428,56.817,0.691233,11.782188,1,1,True,4.645117,-74.061503,1,4,1,1,1
1,77.9,61.0,49.105,3.420,52.525,0.691233,11.782188,1,2,True,4.645117,-74.061503,1,4,1,2,2
2,58.5,48.0,47.284,4.062,51.346,0.691233,11.782188,1,3,True,4.645117,-74.061503,1,4,1,3,3
3,57.2,44.0,46.059,3.606,49.664,0.691233,11.782188,1,4,True,4.645117,-74.061503,1,4,1,4,4
4,53.5,39.0,39.851,1.625,41.475,0.691233,11.782188,1,5,True,4.645117,-74.061503,1,4,1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166435,25.3,18.0,1.404,3.688,5.091,0.264670,29.773000,19,8756,True,4.710381,-74.030414,12,4,31,20,166436
166436,27.7,15.0,0.244,3.205,3.449,0.271380,28.059000,19,8757,True,4.710381,-74.030414,12,4,31,21,166437
166437,25.3,20.0,0.880,4.648,5.529,0.301590,25.561000,19,8758,True,4.710381,-74.030414,12,4,31,22,166438
166438,24.0,17.0,1.697,8.596,10.293,0.349770,20.539000,19,8759,True,4.710381,-74.030414,12,4,31,23,166439


In [36]:
# finalmente se crea un DataFrame, el cual se relaciona con los DataFrame creados anteriormente por medio de sus identificadores 
df_fact_medidad=pd.DataFrame(df, columns=["id_estacion", "id_tiempo", "id_polutante"])
df_fact_medidad

,id_estacion,id_tiempo,id_polutante
0,1,1,1
1,1,2,2
2,1,3,3
3,1,4,4
4,1,5,5
...,...,...,...
166435,19,8756,166436
166436,19,8757,166437
166437,19,8758,166438
166438,19,8759,166439


## 5.6Creación de las tablas en Postgres

In [45]:
# Es necesario tener instaladas las librerias 
#!pip install python-dotenv
#!pip install psycopg2-binary

In [46]:
from conexion import new_model

# se llama a la función new_model (archivo conexion.py) con cada uno de los DataFrame para crear las tablas correspondientes
df_fact_medidad = df_fact_medidad.astype(int)
new_model(df_estacion, "dim_estacion")

#Cree las tablas para dim_polutante, dim_polutante, fact_medidad

